# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [1]:
import pandas as pd
import numpy as np

To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [2]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules
# if IN_COLAB:
#     !wget https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz -O data.tgz
#     !tar -xzvf data.tgz

## How many rows are there in the GBvideos.csv after removing duplications?

In [8]:
gb_df = pd.read_csv('data/GBvideos.csv')
# print(gb_df.shape)
gb_df.drop_duplicates(inplace=True)
print(gb_df.shape)

(38916, 16)
(38745, 16)


In [4]:
# gb_df.head(1)

## How many VDO's that contain at least one record (row) of "dislikes" more than "likes"?  <font color=red>DO NOT group by the data and make sure that you count only unique title!</font>

In [9]:
# gb_df[gb_df['title'] == '#KIISCampus Surprise with BTS | On Air with Ryan Seacrest']
len(gb_df[gb_df['likes']<gb_df['dislikes']]['title'].unique())

56

## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

In [101]:
gb_df['trending_dt'] = pd.to_datetime(gb_df.trending_date, format='%y.%d.%m', errors='ignore', utc=True)
print(len(gb_df[(gb_df['trending_dt'] == '2018-01-22') & (gb_df['comment_count']>10000)]))
# gb_df[(gb_df['trending_dt'] == '2018-01-22') & (gb_df['comment_count']>10000)].sort_values(by=['title'])

28


## Which date that has the minimum average number of comments per VDO?

In [104]:
# gb_df.sort_values(by=['title','trending_date'], ascending=[True, True])
# gb_df[['trending_dt', 'title', 'comment_count']]
day_comment_sum = gb_df[['trending_date', 'comment_count']].groupby('trending_date').sum().reset_index()
day_comment_sum[day_comment_sum['comment_count'] == day_comment_sum['comment_count'].min()]['trending_date'].iloc[0]
# strftime = string format time ex. strftime('%Y-%m-%d')

'17.15.11'

## Compare "Sports" and "Comady", how many days that there are more total daily views of VDO in "Sports" category than in "Comady" category?

In [46]:
import json
with open('data/US_category_id.json') as fd:
    cat = json.load(fd)
# cat
cat_list = []
for d in cat['items']:
    cat_list.append((d['id'], d['snippet']['title']))
# cat_list
cat_df = pd.DataFrame(cat_list, columns=['id', 'category'])
# cat_df.dtypes
cat_df['id'] = cat_df['id'].astype(int)
# cat_df.dtypes
vdo_df_withcat = gb_df.merge(cat_df, left_on='category_id', right_on='id')
# vdo_df_withcat

In [100]:
# print(vdo_df_withcat['trending_date'])
comedy_views = vdo_df_withcat[['trending_dt', 'views']][vdo_df_withcat['category'] == 'Comedy'].groupby(['trending_dt']).sum()
comedy_views.reset_index(inplace=True)
comedy_views.rename(columns={'views': 'c_views'}, inplace=True)
# comedy_views.loc[-1] = [pd.to_datetime('16.14.11', format='%y.%d.%m', errors='ignore', utc=True), 1]  # adding a row
# comedy_views.index = comedy_views.index + 1  # shifting index
# comedy_views = comedy_views.sort_index()  # sorting by index
# print(comedy_views.dtypes)
sports_views = vdo_df_withcat[['trending_dt', 'views']][vdo_df_withcat['category'] == 'Sports'].groupby(['trending_dt']).sum()
sports_views.reset_index(inplace=True)
sports_views.rename(columns={'views': 's_views'}, inplace=True)
# print(sports_views.dtypes)
merged_df = comedy_views.merge(sports_views, on='trending_dt', how='outer')
merged_df.fillna({'c_views': 0, 's_views': 0}, inplace=True)
merged_df = merged_df.astype({'c_views': int, 's_views': int})
# print(merged_df.dtypes)
print((merged_df['s_views'] > merged_df['c_views']).sum())

49
